In [1]:
%matplotlib inline
import os
import time
import numpy as np
from matplotlib import pyplot as plt

from caen_felib import lib, device, error

# Get CAEN FELib path
print(f'CAEN FELib found at: {lib.path} (version={lib.version})')

CAEN FELib found at: libCAEN_FELib.so (version=1.3.2)


In [2]:
from lgdo import lh5, Table, Array, WaveformTable, ArrayOfEqualSizedArrays

## Connect

In [3]:
dig = device.connect("dig2://caendgtz-usb-66154")
print(f'Connected to digitizer (handle={hex(dig.handle)}, name={dig.name})')

Connected to digitizer (handle=0xcae0000000000000, name=)


In [4]:
tree = dig.get_device_tree()

In [11]:
tree["ch"]["0"]["par"].keys()

dict_keys(['adctovolts', 'adcvetowidth', 'anticoincidencemask', 'channelstriggermask', 'channelvetosource', 'chdeadtimemonitor', 'chenable', 'chpretriggers', 'chpretriggert', 'chrealtimemonitor', 'chrecordlengths', 'chrecordlengtht', 'chsavedeventcnt', 'chstatus', 'chtriggercnt', 'chwavecnt', 'coincidencelengths', 'coincidencelengtht', 'coincidencemask', 'dcoffset', 'energy_nbit', 'energyfilterbaselineavg', 'energyfilterbaselineguards', 'energyfilterbaselineguardt', 'energyfilterfinegain', 'energyfilterflattops', 'energyfilterflattopt', 'energyfilterlflimitation', 'energyfilterpeakingavg', 'energyfilterpeakingposition', 'energyfilterpileupguards', 'energyfilterpileupguardt', 'energyfilterpolezeros', 'energyfilterpolezerot', 'energyfilterrisetimes', 'energyfilterrisetimet', 'energyskimhighdiscriminator', 'energyskimlowdiscriminator', 'eventselector', 'eventtriggersource', 'gainfactor', 'gpifunction', 'handle', 'itlconnect', 'pulsepolarity', 'selftrgrate', 'selftriggerwidth', 'signaloffs

In [31]:
#for par, par_dict in tree["par"].items():
for par, par_dict in tree["ch"]["0"]["par"].items():
    if isinstance(par_dict, dict) and "accessmode" in par_dict:
        if par_dict['accessmode']['value'] == "READ_ONLY": continue
        alv = []
        if 'allowedvalues' in par_dict:
            allowedvalues = par_dict['allowedvalues']
            for a in allowedvalues:
                if isinstance(allowedvalues[a], dict):
                    alv.append(allowedvalues[a]["value"])
        elif "minvalue" in par_dict:
            alv = f"From {par_dict['minvalue']['value']} to {par_dict['maxvalue']['value']}, steps of {par_dict['increment']['value']}"

        def_val = ""
        if 'defaultvalue' in par_dict:
            def_val = par_dict['defaultvalue']['value']
        print(f"    {par}: {def_val} # ({par_dict['datatype']['value']}) [{alv}] {par_dict['description']['value']}")

    adcvetowidth: 0 # (NUMBER) [From 0 to 524280, steps of 8] ADC Veto width in nanoseconds. It allows to set a protection time after the ADC saturation. In case the ADC saturation generates an inhibit, the event is not saved; in case the ADC saturation does not generate an inhibit, the triggers generated during the veto width are flagged.
    anticoincidencemask: Disabled # (STRING) [['Disabled', 'Ch64Trigger', 'TRGIN', 'GlobalTriggerSource', 'ITLA', 'ITLB']] AntiCoincidence Mask
    channelstriggermask: 0 # (STRING) [[]] Defines the trigger source (or the coincidence source) for the Time-Energy Events
    channelvetosource: Disabled # (STRING) [['Disabled', 'BoardVeto', 'ADCOverSaturation', 'ADCUnderSaturation']] This parameter defines the source of the Channel Veto
    chenable: True # (STRING) [['True', 'False']] Enable channel for the acquisition. When the channel is disabled, it doesn't give any data and its self-trigger is off.
    chpretriggers: 250 # (NUMBER) [From 4 to 4000, 

In [ ]:
# Reset
dig.cmd.Reset()

fw_type = dig.par.fwtype.value
fw_ver = dig.par.fpga_fwver.value
print(fw_type, fw_ver)

adc_samplrate_msps = float(dig.par.adc_samplrate.value)  # in Msps
adc_n_bits = int(dig.par.adc_nbit.value)
sampling_period_ns = int(1e3 / adc_samplrate_msps)

print(f"Sampling rate = {adc_samplrate_msps} MHz, n. bit = {adc_n_bits}, Sampling period = {sampling_period_ns} ns")

# Configuration parameters
nch = int(dig.par.NumCh.value)
active_ch = 2
reclen = 8000 #sample
pretrg = 4000 #sample
dc_offset = 10

dig.par.iolevel.value = "TTL"
dig.par.globaltriggersource.value = "TrgIn"

# Set some channel parameters
for i, ch in enumerate(dig.ch):
    ch.par.chenable.value = "TRUE" if i < active_ch else "FALSE"  # Enable only channel 0
    ch.par.eventtriggersource.value = "GLOBALTRIGGERSOURCE"
    ch.par.wavetriggersource.value = "GLOBALTRIGGERSOURCE"
    ch.par.chrecordlengths.value = f"{reclen}"
    ch.par.chpretriggers.value = f"{pretrg}"
    ch.par.waveanalogprobe0.value = "ADCINPUT"
    ch.par.wavedigitalprobe0.value = "TRIGGER"
    ch.par.wavedigitalprobe1.value = "ADCSaturation"
    ch.par.dcoffset.value = f"{dc_offset}"

## Configure endpoint

In [ ]:
data_format = [
    {
        'name': 'CHANNEL',
        'type': 'U8',
        'dim' : 0
    },
    {
        'name': 'TIMESTAMP',
        'type': 'U64',
        'dim': 0,
    },
    {
        'name': 'ENERGY',
        'type': 'U16',
        'dim': 0,
    },
    {
        'name': 'ANALOG_PROBE_1',
        'type': 'U16',
        'dim': 1,
        'shape': [reclen]
    },
    {
        'name': 'ANALOG_PROBE_1_TYPE',
        'type': 'I32',
        'dim': 0
    },
    {
        'name': 'DIGITAL_PROBE_1',
        'type': 'U8',
        'dim': 1,
        'shape': [reclen]
    },
    {
        'name': 'DIGITAL_PROBE_1_TYPE',
        'type': 'I32',
        'dim': 0
    },
    {
        'name': 'DIGITAL_PROBE_2',
        'type': 'U8',
        'dim': 1,
        'shape': [reclen]
    },
    {
        'name': 'DIGITAL_PROBE_2_TYPE',
        'type': 'I32',
        'dim': 0
    },
    {
        'name': 'WAVEFORM_SIZE',
        'type': 'SIZE_T',
        'dim': 0
    }
]

In [5]:
decoded_endpoint_path = "dpppha"
endpoint = dig.endpoint[decoded_endpoint_path]
data = endpoint.set_read_data_format(data_format)
dig.endpoint.par.activeendpoint.value = decoded_endpoint_path

channel = data[0].value
ts = data[1].value
energy = data[2].value
analog_probe_1 = data[3].value
analog_probe_1_type = data[4].value  # Integer value described in Supported Endpoints > Probe type meaning
digital_probe_1 = data[5].value
digital_probe_1_type = data[6].value  # Integer value described in Supported Endpoints > Probe type meaning
digital_probe_2 = data[7].value
digital_probe_2_type = data[8].value  # Integer value described in Supported Endpoints > Probe type meaning
waveform_size = data[9].value

In [6]:
nev = 10
timestamp = np.zeros((active_ch,nev),dtype=np.uint64)
wfs = np.zeros((active_ch,nev,reclen),dtype=np.uint16)
temp_names = ["tempsensfirstadc","tempsenshottestadc","tempsenslastadc","tempsensairin","tempsensairout","tempsenscore","tempsensdcdc"]
temperatures = np.zeros((nev,len(temp_names)),dtype=float)

In [ ]:
for i in dig.par.child_nodes:
    print(f'type: {i.type.name}\tname:{i.name}')
    for j in i.child_nodes:
        print(f'\t{j.type.name}\t{j.name} = {j.value}')

In [ ]:
dig.cmd.armacquisition()
dig.cmd.swstartacquisition()

for i in range(nev*active_ch):
    #dig.cmd.sendswtrigger()
    try:
        endpoint.read_data(-1, data)
        wfs[channel,i//active_ch] = analog_probe_1
        timestamp[channel,i//active_ch] = ts
        for j, temp in enumerate(temp_names):
            temp_value = float(dig.get_value(f"/par/{temp}"))
            temperatures[i//active_ch][j] = temp_value
    except error.Error as ex:
        if ex.code == error.ErrorCode.TIMEOUT:
            continue
        if ex.code == error.ErrorCode.STOP:
            break
        else:
            raise ex
dig.cmd.disarmacquisition()

In [ ]:
raw_data = lh5.read("ch1/raw",f"/home/dedeo/test_CAEN/data_V2740/20240730/prova_000.lh5")
wfs = raw_data.waveform.values.nda
timestamp = raw_data.timestamp.nda
#temp0 = raw_data.temp0.nda
print(raw_data.keys())

In [ ]:
fig, ax = plt.subplots(figsize=(12,6.75))
for i in range(5):
    ax.plot(wfs[i],label=f"timestamp = {timestamp[i]}")
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6.75))
#ax.plot(voltages,wf_max,"b.")
pars = np.polyfit(voltages,wf_max,1)
ax.plot(wf_max,wf_max-np.polyval(pars,voltages),"b.")
ax.axhline(0,color="k",ls=":")
ax.set_ylabel("Residuals (LSB)")
ax.set_xlabel("ADC channels (LSB)")

In [ ]:
12 * 60 * 60

In [ ]:
750*432

In [ ]:
100000/60/60